<a href="https://colab.research.google.com/github/cmarta22/Diagnostico_tardio_de_VIH_TDF/blob/main/Dataset_VIH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importo de los  Dataset desde Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Union de los 2 dataset

In [42]:
import pandas as pd

# Definir la ruta de drive
path = "/content/drive/MyDrive/Dataset_Originales"

# Leer el archivo CSV de Dignostico tardio
df_dx = pd.read_csv(path + "/dx-tardio-vih-genero-grupo-etario-2010-2019-argentina_1.csv", encoding='latin1')

# Leer el archivo CSV de tasa de VIH
df_tasa = pd.read_csv(path + "/tasa-vih-por-100-mil-habitantes-sexo-jurisdiccion-2010-2022-argentina-actualizacion_1.csv",encoding='latin1')






In [51]:
# Ver columnas para elegir con cual see puede unir
print("Columnas df_dx:", df_dx.columns)
print("Columnas df_tasa:", df_tasa.columns)

Columnas df_dx: Index(['id_genero', 'sexo', 'id_grupo_etario', 'grupo_etario', 'bienio',
       'porcentaje_diagnostico_tardio'],
      dtype='object')
Columnas df_tasa: Index(['id_sexo;sexo;id_jurisdiccion;jurisdiccion;anio;jurisdiccion_tasa_vih'], dtype='object')


In [46]:
# Elimina espacio invisibles y mayusculas/minusculas
df_dx.columns = df_dx.columns.str.strip().str.lower()
df_tasa.columns = df_tasa.columns.str.strip().str.lower()

In [54]:
print(df_dx.columns)
print(df_tasa.columns)

Index(['id_genero', 'sexo', 'id_grupo_etario', 'grupo_etario', 'bienio',
       'porcentaje_diagnostico_tardio'],
      dtype='object')
Index(['id_sexo', 'sexo', 'id_jurisdiccion', 'jurisdiccion', 'anio',
       'jurisdiccion_tasa_vih'],
      dtype='object')


In [55]:
path = "/content/drive/MyDrive/Dataset_Originales/"  # Concatenar bien la ruta del archivo

df_tasa = pd.read_csv(path + "tasa-vih-por-100-mil-habitantes-sexo-jurisdiccion-2010-2022-argentina-actualizacion_1.csv",
                      encoding='latin1', sep=';')

In [60]:
# Paso 1: Separar los años del bienio
df_dx[['anio_inicio', 'anio_fin']] = df_dx['bienio'].str.split('-', expand=True).astype(int)

# Paso 2: Expandir cada fila por los años del bienio
lista_filas = []

for _, row in df_dx.iterrows():
    for anio in range(row['anio_inicio'], row['anio_fin'] + 1):
        lista_filas.append({
            'anio': anio,
            'sexo': row['sexo'],
            'grupo_etario': row['grupo_etario'],
            'porcentaje_diagnostico_tardio': row['porcentaje_diagnostico_tardio']
        })

df_dx_expandido = pd.DataFrame(lista_filas)

# Paso 3: Filtrar entre 2010 y 2019
df_dx_expandido = df_dx_expandido[df_dx_expandido['anio'].between(2010, 2019)]

# Verificá que la columna 'anio' exista ahora
print(df_dx_expandido.columns)
print(df_dx_expandido.head())




Index(['anio', 'sexo', 'grupo_etario', 'porcentaje_diagnostico_tardio'], dtype='object')
   anio          sexo grupo_etario  porcentaje_diagnostico_tardio
0  2013  Total_genero  total_grupo                           24.9
1  2014  Total_genero  total_grupo                           24.9
2  2015  Total_genero  total_grupo                           30.9
3  2016  Total_genero  total_grupo                           30.9
4  2017  Total_genero  total_grupo                           31.1


In [61]:
# Filtrar entre 2010 y 2019
df_dx_expandido = df_dx_expandido[df_dx_expandido['anio'].between(2010, 2019)]

# Hacer la union por sexo y año
df_merged = pd.merge(df_dx_expandido, df_tasa, on=['sexo', 'anio'], how='inner')

# Ver primeras filas del resultado
print(df_merged.head())

Empty DataFrame
Columns: [anio, sexo, grupo_etario, porcentaje_diagnostico_tardio, id_sexo, id_jurisdiccion, jurisdiccion, jurisdiccion_tasa_vih]
Index: []


In [63]:
# Para ver cómo evolucionó el diagnóstico tardío y la tasa de VIH en distintos rangos etarios
df_por_grupo = df_merged.groupby(['grupo_etario', 'anio']).agg({
    'porcentaje_diagnostico_tardio': 'mean',
    'jurisdiccion_tasa_vih': 'mean'
}).reset_index()


In [65]:
#Agrupar por jurisdiccion y año
df_por_jurisdiccion = df_merged.groupby(['jurisdiccion', 'anio']).agg({
    'porcentaje_diagnostico_tardio': 'mean',
    'jurisdiccion_tasa_vih': 'mean'
}).reset_index()


In [66]:
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   anio                           0 non-null      int64  
 1   sexo                           0 non-null      object 
 2   grupo_etario                   0 non-null      object 
 3   porcentaje_diagnostico_tardio  0 non-null      float64
 4   id_sexo                        0 non-null      int64  
 5   id_jurisdiccion                0 non-null      int64  
 6   jurisdiccion                   0 non-null      object 
 7   jurisdiccion_tasa_vih          0 non-null      object 
dtypes: float64(1), int64(3), object(4)
memory usage: 132.0+ bytes


In [68]:
df_merged.to_csv("/content/drive/MyDrive/Dataset_Unificado.csv", index=False)